# Machine Learning for Seagrass Extent Prediction

In [254]:
from pystac.client import Client
from odc.stac import load
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import joblib
from matplotlib import colors
from sklearn.ensemble import RandomForestClassifier 


from utils_test import scale, do_prediction, calculate_band_indices, apply_masks

## Loading Sentinel-2 GeoMAD

Load data, then create band indices and mask out areas we don't want to include.

In [255]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [256]:
#version = "funafuti_mask_test4"

In [257]:
gdf = gpd.read_file("training-data/bootless_bay.geojson")
#version = "malekula_joined_model_mask_test3"
#gdf = gpd.read_file("training-data/Alluvium_vu_malekula_13022025.geojson")
gdf = gdf.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
bbox = [min_lon, min_lat, max_lon, max_lat]
datetime="2024"

In [258]:
items = client.search(
    collections=["dep_s2_geomad"], datetime=datetime, bbox=bbox
).item_collection()

print(f"Found {len(items)} items")

Found 1 items


In [259]:
data = load(
    items,
    bbox=bbox,
    measurements=[
        "nir",
        "red",
        "blue",
        "green",
        "emad",
        "smad",
        "bcmad",
        "count",
        "green",
        "nir08",
        "nir09",
        "swir16",
        "swir22",
        "coastal",
        "rededge1",
        "rededge2",
        "rededge3",
    ],
    chunks={"x": 2048, "y": 2048},
)

data

<xarray.Dataset> Size: 5MB
Dimensions:      (y: 490, x: 262, time: 1)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    red          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    blue         (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    green        (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    emad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    smad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    swir22       (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    coastal      (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    rededge1     (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    rededge2     (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    rededge3     (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>

# Before Masking 

In [260]:
scaled_data = scale(data)
indices = calculate_band_indices(scaled_data)

scaled_data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

# Masking

In [261]:
# scaled_data = scale(data)
#indices = calculate_band_indices(scaled_data)

#masked_data = apply_masks(scaled_data)

#masked_data
# drop stumpf and ln_bg
#masked_data = masked_data.drop_vars(['stumpf', 'ln_bg'])


In [262]:
# data = scaled_data

# #Land Mask 0,-0.35,-0.5

# mndwi = (data["green"] - data["swir16"]) / (data["green"] + data["swir16"])
# mndwi_land_mask = mndwi > -0.5
# masked_data = data.where(mndwi_land_mask)

# masked_data = scaled_data
# #turbidity mask 0.2,0,-0.001,0.1 

# ndti = (masked_data["red"] - masked_data["green"]) / (
#     masked_data["red"] + masked_data["green"]
#     )
# ndti_mask = ndti < 0.2
# masked_data = masked_data.where(ndti_mask)


# #surf mask 

# nir_mask = masked_data["nir"] < 0.085
# masked_data = masked_data.where(nir_mask)

# # Deepsea mask 
# # Natural log of blue/green
# masked_data["ln_bg"] = np.log(masked_data.blue / masked_data.green)
# bg = masked_data["ln_bg"]

# # deep sea mask   minor = 0.1 & 0.2 | moderate = 0.0000001| major = -0.0001 
# mask_bg = bg < -0.0001
# masked_data = masked_data.where(mask_bg)

# # drop
# # masked_data = masked_data.drop_vars(['stumpf', 'ln_bg'])

In [326]:
# #Deep Sea mask exclusive 
# # This cell is used to run deep sea mask seperately (without out other mask) to see changes, when configuring threshhold 
# data = scaled_data

# # Deepsea mask 
# # Natural log of blue/green
# data["ln_bg"] = np.log(data.blue / data.green)
# bg = data["ln_bg"]

# # deep sea mask   minor = 0.1 & 0.2 | moderate = 0.0000001| major = -0.001 
# mask_bg = bg <  0.2
# ds_mask  = data.where(mask_bg)



In [331]:
#Threshold functions (still in progress)

# data["ln_bg"] = np.log(data.blue / data.green)
# bg = data["ln_bg"]

# #mean 
# mean_bg = data["ln_bg"].mean().compute().item()  # .item() gets the Python scalar
# #sd
# std_bg = data["ln_bg"].std().compute().item()

# thresh_moderate = std_bg
# thresh_minor = mean_bg - std_bg
# thresh_major = mean_bg + std_bg



# print("minor", thresh_minor, "moderate", thresh_moderate, "major", thresh_major)
# print("Mean", mean_bg)
# print("Sd", std_bg)





In [332]:
#Cell used for printing min and max values 
#print("Min:", data["nir"].min())
#print("Max:", data["nir"].max())

print("Min:", data["ln_bg"].min().compute().item())
print("Max:", data["ln_bg"].max().compute().item())

Min: -1.0307010830518994
Max: 0.35765485582508116


In [329]:
# #Re-arranged Mask 
# data = scaled_data

# masked_data = scaled_data

# #surf mask 

# nir_mask = masked_data["nir"] < 0.085
# masked_data = masked_data.where(nir_mask)

# #turbidity mask 0.2,0,-0.001,0.1 

# ndti = (masked_data["red"] - masked_data["green"]) / (
#     masked_data["red"] + masked_data["green"]
#     )
# ndti_mask = ndti < 0.2
# masked_data = masked_data.where(ndti_mask)

# #Land Mask 0,-0.35,-0.5

# mndwi = (data["green"] - data["swir16"]) / (data["green"] + data["swir16"])
# mndwi_land_mask = mndwi > -0.5
# masked_data = data.where(mndwi_land_mask)

# # Natural log of blue/green
# data["ln_bg"] = np.log(data.blue / data.green)
# bg = data["ln_bg"]
# # conservative deep sea mask
# mask_bg = bg < 0.0000001

# ds_mask = data.where(mask_bg)


# #drop
#masked_data = masked_data.drop_vars(['stumpf', 'ln_bg'])

In [330]:
ds_mask.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

In [224]:
#masked_data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

In [211]:
masked_data = masked_data.drop_vars(['stumpf', 'ln_bg'])
masked_data

<xarray.Dataset> Size: 204MB
Dimensions:      (time: 1, y: 1142, x: 952)
Coordinates:
  * y            (y) float64 9kB -1.842e+06 -1.842e+06 ... -1.854e+06 -1.854e+06
  * x            (x) float64 8kB 1.978e+06 1.978e+06 ... 1.988e+06 1.988e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/25)
    nir          (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    red          (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    blue         (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    green        (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    emad         (time, y, x) float32 4MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    smad         (time, y, x) float32 4MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    ...           ...
    ndvi         (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    ndwi         (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    b_g          (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    b_r          (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    mci          (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>
    ndci         (time, y, x) float64 9MB dask.array<chunksize=(1, 1142, 952), meta=np.ndarray>

# Run predictions over our region

In [212]:
model = joblib.load("models/model-geomad-joined-data-rf-11042025.model")

In [213]:
loaded = masked_data.compute()

In [214]:
loaded

<xarray.Dataset> Size: 204MB
Dimensions:      (time: 1, y: 1142, x: 952)
Coordinates:
  * y            (y) float64 9kB -1.842e+06 -1.842e+06 ... -1.854e+06 -1.854e+06
  * x            (x) float64 8kB 1.978e+06 1.978e+06 ... 1.988e+06 1.988e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/25)
    nir          (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    red          (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    blue         (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    green        (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    emad         (time, y, x) float32 4MB nan nan nan nan ... nan nan nan nan
    smad         (time, y, x) float32 4MB nan nan nan nan ... nan nan nan nan
    ...           ...
    ndvi         (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    ndwi         (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    b_g          (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    b_r          (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    mci          (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan
    ndci         (time, y, x) float64 9MB nan nan nan nan ... nan nan nan nan

In [215]:
predicted_da = do_prediction(loaded, model).astype(np.float32)
predicted_da

<xarray.DataArray (y: 1142, x: 952, time: 1)> Size: 4MB
array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
...
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 9kB -1.842e+06 -1.842e+06 ... -1.854e+06 -1.854e+06
  * x            (x) float64 8kB 1.978e+06 1.978e+06 ... 1.988e+06 1.988e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01

# Feature Importance

In [48]:
# gdf = gpd.read_file("training-data/Alluvium_vu_malekula_13022025.geojson")
# gdf = gdf.to_crs("EPSG:4326")
# min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
# bbox = [min_lon, min_lat, max_lon, max_lat]
# datetime="2024"

In [16]:
# Define training data
#gdf = gpd.read_file("training-data/Alluvium_vu_malekula_13022025.geojson")
#gdf = gdf.to_crs("EPSG:4326")
# gdf.explore(column="cc_id", legend=True)

In [85]:
postcard =data #.where(all_masks)
# postcard =clipped_ds.to_array(dim="band")
# First transform the training points to the same CRS as the data
training = gdf.to_crs(postcard.odc.geobox.crs)

In [86]:
print(training.columns)
#training=training.drop(columns=['date', 'uuid'])

Index(['fid', 'Major_Ecosystem', 'Percentage_Cover1', 'Minor_Ecosystem',
       'Percentage_Cover2', 'Photo', 'Notes', 'Date', 'Surveyor', 'geometry'],
      dtype='object')


In [89]:
training

,fid,Major_Ecosystem,Percentage_Cover1,Minor_Ecosystem,Percentage_Cover2,Photo,Notes,Date,Surveyor,geometry
0,1,mangrove,100,None,None,None,None,2025-03-19 09:37:04.150000+00:00,None,POINT (-301531.722 -1056410.427)
1,2,mangrove,100,mangrove,None,None,None,2025-03-19 09:37:04.150000+00:00,Waqa,POINT (-302035.169 -1056172.465)
2,3,None,None,None,None,None,None,2025-03-19 09:37:04.150000+00:00,Waqa,POINT (-302035.622 -1056163.948)
3,4,mangrove,90,mangrove,None,None,None,2025-03-19 09:37:04.150000+00:00,Waqa,POINT (-301678.701 -1055570.345)
4,5,mangrove,90,None,None,None,None,2025-03-19 09:37:04.150000+00:00,Waqa,POINT (-301676.178 -1055605.274)
...,...,...,...,...,...,...,...,...,...,...
4647,4648,sand,None,seagrass,None,DCIM/geotag-macbluepng_20250319141225512.jpg,None,2025-03-19 01:28:11.744000+00:00,Max Andrew,POINT (-302189.535 -1057825.943)
4648,4649,seagrass,None,sand,None,None,None,2025-03-19 01:28:11.744000+00:00,Max Andrew,None
4649,4650,seagrass,None,seagrass,None,DCIM/geotag-macbluepng_20250319143359132.jpg,None,2025-03-19 01:28:11.744000+00:00,Max Andrew,None
4650,4651,seagrass,None,None,None,DCIM/geotag-macbluepng_20250319143523650.jpg,None,2025-03-19 01:28:11.744000+00:00,Max Andrew,None


In [87]:
training_da = training.assign(x=training.geometry.x, y=training.geometry.y).to_xarray()

In [88]:
training_values = (
    postcard.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)
training_values

,y,x,spatial_ref,time,nir,red,blue,green,emad,smad,bcmad,count,nir08,nir09,swir16,swir22,coastal,rededge1,rededge2,rededge3
index,,,,,,,,,,,,,,,,,,,,
0,-1056415.0,-301535.0,3832,2024-01-01,2619,528,397,737,401.137695,0.000659,0.028749,31,2923,2881,1511,892,315,1206,2361,2690
1,-1056175.0,-302035.0,3832,2024-01-01,1183,540,536,696,960.667786,0.008482,0.143958,21,1091,923,933,701,724,907,1003,1040
2,-1056165.0,-302035.0,3832,2024-01-01,1361,750,656,821,911.694824,0.012277,0.150911,21,1074,906,946,760,715,883,985,1055
3,-1055575.0,-301675.0,3832,2024-01-01,2364,271,285,432,794.167114,0.006631,0.086733,21,2383,1801,805,522,770,739,1827,2219
4,-1055605.0,-301675.0,3832,2024-01-01,2229,287,302,424,906.952942,0.008708,0.102485,20,2177,1678,799,513,799,751,1700,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4647,-1057825.0,-302185.0,3832,2024-01-01,1809,1974,1539,1876,1476.850342,0.010534,0.115055,29,1748,1083,1267,864,872,1811,1628,1753
4648,-1049325.0,-299305.0,3832,2024-01-01,2291,413,378,595,1323.421509,0.005048,0.099559,11,2937,3371,1848,968,350,1004,2214,2617
4649,-1049325.0,-299305.0,3832,2024-01-01,2291,413,378,595,1323.421509,0.005048,0.099559,11,2937,3371,1848,968,350,1004,2214,2617


In [84]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)
# Drop rows where there was no data available
training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

KeyError: 'cc_id'

In [ ]:
postcard_df = training_array
postcard_df.columns

In [ ]:
postcard_df.columns

In [ ]:
# postcard_df.columns
# postcard_df = postcard_df.drop(columns=["spatial_ref", "time", "field_1", "y", "x"])
# postcard_df = postcard_df.drop(columns=["field_1"])
# postcard_df = postcard_df.drop(columns=["y", "x", "spatial_ref"])

postcard_df = postcard_df.drop(columns=["time"])
#postcard_df

In [ ]:
postcard_df

In [ ]:
len(postcard_df.columns)

In [60]:
# The classes are the first column
classes = np.array(postcard_df)[:, 0]

# The observation data is everything after the second column
observations = np.array(postcard_df)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [63]:
postcard_params = postcard_df.columns[1:]
print(postcard_params)

Index(['y', 'x', 'spatial_ref', 'nir', 'red', 'blue', 'green', 'emad', 'smad',
       'bcmad', 'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal',
       'rededge1', 'rededge2', 'rededge3'],
      dtype='object')


In [64]:


importances = classifier.feature_importances_

# Create the DataFrame
data = {
    'param': postcard_params,  # Use the list directly
    'importance': importances,  # Use the list directly
}
importance_df = pd.DataFrame(data)

importance_df["%"] = (importance_df["importance"]*100)

sorted_df = importance_df.sort_values(by='importance', ascending=False)
sorted_df.to_csv(f"{version}importance_df.csv")

sorted_df
# sorted_df.to_csv(.csv', index=False)

,param,importance,%
0,y,0.110710,11.070960
13,swir16,0.104350,10.434962
14,swir22,0.092492,9.249163
3,nir,0.081210,8.120965
1,x,0.060902,6.090236
17,rededge2,0.059974,5.997446
12,nir09,0.056494,5.649439
16,rededge1,0.054797,5.479676
11,nir08,0.054460,5.446040
18,rededge3,0.054371,5.437092


In [65]:
sorted_df


,param,importance,%
0,y,0.110710,11.070960
13,swir16,0.104350,10.434962
14,swir22,0.092492,9.249163
3,nir,0.081210,8.120965
1,x,0.060902,6.090236
17,rededge2,0.059974,5.997446
12,nir09,0.056494,5.649439
16,rededge1,0.054797,5.479676
11,nir08,0.054460,5.446040
18,rededge3,0.054371,5.437092


# Visualisation
Visualise the prediction with the 8 class colour map or the 10 class colour map below, respectively

In [216]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [8]
norm = colors.BoundaryNorm(bounds, c_map.N)
predicted_da.odc.explore(cmap=c_map)

In [217]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#00FFFFFF"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

In [218]:
predicted_da.odc.explore(cmap=c_map)

In [220]:
version = "malekula_ds_mask_test_dsonly_minor"

In [221]:
# predicted_da.odc.write_cog("predictions/predicted_ba_estuary_joined_data_postcard_svm_22042025.tiff", overwrite=True)
#predicted_da.odc.write_cog("predictions/predicted_komave_postcard_rf_24042025.tiff", overwrite=True)
predicted_da.odc.write_cog(f"predictions/{version}.tiff", overwrite=True)

PosixPath('predictions/malekula_ds_mask_test_major_00.tiff')

In [222]:
predicted_da = predicted_da.where(bg < 0.2)
predicted_da.odc.explore(cmap=c_map, bands = ["red","blue","green"], vmax =0.3, vmin =0)


In [129]:
#predicted_da.rio.to_raster(f"predictions/{version}.tiff")
predicted_da_band = predicted_da.squeeze(drop=True)
predicted_da_band.rio.to_raster("output2.tiff")

# Quantitative validation (to be completed)

In [ ]:
test_data = gpd.read_file("testing-data/utanglang_postcard.geojson")
test_data.explore()

In [ ]:
print("stop here")

In [ ]:
stop_here


In [ ]:
# First transform the training points to the same CRS as the data
test = test_data.to_crs(postcard.odc.geobox.crs)
# Next get the X and Y values out of the point geometries
training_da = test.assign(x=test.geometry.x, y=test.geometry.y).to_xarray()
# Now we can use the x and y values (lon, lat) to extract values from the median composite
training_values = (
    postcard.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)
len(training_values)

# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([test["observed_id"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)
# # Drop rows where there was no data available
# training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

In [ ]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([test["observed_id"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)
# # Drop rows where there was no data available
# training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

In [ ]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

In [ ]:
print(len(training_array), len(test))  # Check the lengths of both arrays

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Sample data
np.random.seed(42)
training_array = np.random.rand(100, 5)
test = pd.DataFrame({"observed_id": np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 100)})

# Split the data
X_train, X_test, y_train, y_test = train_test_split(training_array, test.observed_id, test_size=0.9, random_state=42)

# Replace None values with a default value, e.g., 0 or the most frequent value
y_train = y_train.fillna(0)  # or y_train.fillna(y_train.mode()[0])
y_test = y_test.fillna(0)    # Ensure y_test also has no None values
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Train your model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Generate and display confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)
plt.figure(figsize=(12, 10))  # Adjust width and height for larger plot
cm_display.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
cm

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import classification_report

# report = classification_report(true_labels, predicted_labels, target_names=class_labels)
# print(report)

report = classification_report(y_test, y_pred, target_names=['sediment', 'sand', 'rubble', 'seagrass', 'seaweed', 'coral', 'rock', 'deeps', 'mangrove', 'land'])
print(report)

In [ ]:
print("Unique labels in y_test:", np.unique(y_test))
print("Unique labels in y_pred:", np.unique(y_pred))

In [ ]:
print("Length of y_test:", len(y_test))
print("Length of y_pred:", len(y_pred))